In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
from helpers import *

In [4]:
# path to project directory
path = Path('./')

In [5]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v4.csv', index_col=0, dtype={'season':str})

A baseline model is an effective way to see whether more sophisticated approaches actually have more predictive power.

The approach we will take is very straightforward: a player's predicted score is simply the points scored per minute for every minute played in history, multiplied by the actual number of minutes played in the gameweek being predicted (note that, at the moment, the predicted number of minutes is not included in the model).

We'll start by using the player_lag_features function to obtain the points per game (90 minutes) for all games in history at each gameweek for each player.

In [6]:
# add all lag points scored to training set
train_df, player_lag_vars = player_lag_features(train_df, ['total_points'], ['all'])

Our baseline model function generates predictions using the above approach for a given validation point (i.e. which gameweek the validation starts) and validation length (how many gameweeks will we include, from the validation point). If a player has no history then we just assume that they will score 0 points.

We return the predictions and actuals to allow easy performance assessment.

In [15]:
def simple_model(df, valid_season, valid_gw, valid_len):
    valid_start, valid_end = validation_season_idx(df, valid_season, [valid_gw], valid_len)[0]
    
    train_idx = range(valid_start)
    valid_idx = range(valid_start, valid_end + 1)    
    
    train = df.iloc[train_idx]
    valid = df[['player', 'gw', 'season', 'minutes', 'total_points']].iloc[valid_idx]
    
    season_point = valid['season'].iloc[0]
    gw_point = valid['gw'].iloc[0]
    
    # get player total per game average at validation point
    player_points_pg = df[(df['season'] == season_point) & 
                          (df['gw'] == gw_point)][['player', 'total_points_pg_last_all']]
    
    pred_df = valid.merge(player_points_pg, on='player', how='left')   
    pred_df.fillna(0, inplace=True)
    preds = pred_df['total_points_pg_last_all'] * pred_df['minutes'] / 90
    targs = pred_df['total_points']
    
    return preds, targs

We can ahead and use this model to make some predictions.

In [16]:
# set validation start at gameweek 20 in the 2019/20 season
# validate 6 gameweek's of predictions
valid_season = '1920'
valid_gw = 20
valid_len = 6

# make predictions
preds, targs = simple_model(train_df, valid_season, valid_gw, valid_len)

And calculate the root mean squared error for all predictions

In [17]:
# calculate rsme
r_mse(preds, targs)

1.904038

It's worth running little tests to make check for errors. In this case we can perform the same prediction using a slightly more direct way (not using any indexes).

In [18]:
# we can calculate the same predictions directly using the season and gameweek
# to check the above function 
# get player total per game average at validation point
player_points_pg = train_df[(train_df['season'] == '1920') & 
                            (train_df['gw'] == valid_gw)][['player', 'total_points_pg_last_all']]

valid_df = train_df[(train_df['season'] == '1920') 
                    & (train_df['gw'] > valid_gw - 1)
                    & (train_df['gw'] < valid_gw + 6)][['player', 'minutes', 'total_points']]

pred_df = valid_df.merge(player_points_pg, on='player', how='left')
pred_df['pred'] = pred_df['total_points_pg_last_all'] * pred_df['minutes'] / 90
pred_df.fillna(0, inplace=True)

In [19]:
# calculate rmse
r_mse(pred_df['pred'], pred_df['total_points'])

1.904038

The two error values should equal. The intention is to introduce more of these tests in a more robust software engineering way (i.e. unit tests) in the future.

In the above model we use the lag points per game values from the first validation gameweek, since these contain information for everything up to but not including the validation gameweek. Since these lag variables are calculated for each gameweek, it is important not to use them for predictions being made more than 1 gameweek after the validation point. The above model gives us a useful way to create a new training dataset that avoids this problem, and just sets lag features to be equal to the first valition gameweek for all subsequent gameweeks.

This can then be used to train and validate some more sophisticated modeling.

In [3]:
# We can adapt this approach to also create validation sets with lag features
# When making predictions for gw +2 and beyond we cannot use those weeks's lag features
# This would be leakage if we did
# Instead, each subsequent validation week should have the same lag values as the first
def create_lag_train(df, cat_vars, cont_vars, lag_vars, dep_var, valid_season, valid_gw, valid_len):

    # get all the lag data for the current season up to the first validation gameweek
    player_lag_vals = df[(df['season'] == valid_season) & 
                         (df['gw'] <= valid_gw)][['player', 'gw'] + lag_vars]
    
    # get the last avaialable lag data for each player
    # for most it will be the first validation week
    # but sometimes teams have blank gameweeks
    # in these cases it will be the previous gameweek
    player_lag_vals = player_lag_vals[player_lag_vals['gw'] == player_lag_vals.groupby('player')['gw'].transform('max')]
    player_lag_vals = player_lag_vals.drop('gw', axis=1)

    # get the validation start and end indexes
    valid_start, valid_end = validation_season_idx(df, valid_season, [valid_gw], valid_len)[0]
    train_idx = range(valid_start)
    valid_idx = range(valid_start, valid_end + 1)    

    # split out train and validation sets
    # do not include lag vars in validation set
    train = df[['player'] + cat_vars + cont_vars + lag_vars + dep_var].iloc[train_idx]
    valid = df[['player'] + cat_vars + cont_vars + dep_var].iloc[valid_idx]

    # add in lag vars
    # will be the same for all validation gameweeks
    valid = valid.merge(player_lag_vals, on='player', how='left')
    
    # concatenate train and test again
    lag_train_df = pd.concat([train, valid]).reset_index()

    return lag_train_df, train_idx, valid_idx

The arguments for this function also includes the categorical, continuous and lagging features that we want to include in our model - this fits nicely with the APIs for the algorithms that we will subsequently be applying.

In [22]:
# variables to include
# lag variables are already generated in the player_lag_features function, but can be changed
cat_vars = ['gw', 'season']
cont_vars = ['minutes']
dep_var = ['total_points']

lag_train_df = create_lag_train(train_df, cat_vars, cont_vars, player_lag_vars, valid_season, valid_gw, valid_len)

We can do a quick quick to make sure that the lag variables are the same for gameweeks 20-25

In [26]:
# check features for a player
lag_train_df[lag_train_df['player'] == 'Mohamed_Salah'].tail(10)

,index,player,gw,season,minutes,total_points_pg_last_all,total_points
76699,76699,Mohamed_Salah,16,1920,90,7.996931,13
77163,77163,Mohamed_Salah,17,1920,90,8.058961,16
78202,78202,Mohamed_Salah,19,1920,69,8.156212,3
78784,394,Mohamed_Salah,20,1920,90,8.116743,3
79369,979,Mohamed_Salah,21,1920,90,8.116743,10
79960,1570,Mohamed_Salah,22,1920,90,8.116743,6
80757,2367,Mohamed_Salah,23,1920,90,8.116743,7
81236,2846,Mohamed_Salah,24,1920,84,8.116743,2
81237,2847,Mohamed_Salah,24,1920,90,8.116743,14
81870,3480,Mohamed_Salah,25,1920,90,8.116743,16
